In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.utils import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
import os
from sklearn import metrics

Prepare train and validation set.

In [3]:
train_dir = '/content/drive/MyDrive/Seminar2/data/ribe_512x768/train/'
class_names = {'divlje': 0, 'tunakavez': 1, 'uzgoj': 2}

image_files = []
labels = []

for class_name in list(class_names.keys()):
    class_dir = os.path.join(train_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        image_files.append(np.array(cv.imread(image_path)))
        labels.append(class_names[class_name])

In [4]:
train_files, valid_files, train_labels, valid_labels = train_test_split(
    image_files, labels, test_size=0.33, stratify=labels, random_state=42
)

Prepare test set.

In [5]:
test_dir = '/content/drive/MyDrive/Seminar2/data/ribe_512x768/test/'

test_files = []
test_labels = []

for class_name in list(class_names.keys()):
    class_dir = os.path.join(test_dir, class_name)
    for image_file in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_file)
        test_files.append(np.array(cv.imread(image_path)))
        test_labels.append(class_names[class_name])

In [6]:
X_train = np.array(train_files)
y_train = np.array(train_labels)

X_valid = np.array(valid_files)
y_valid = np.array(valid_labels)

X_test = np.array(test_files)
y_test = np.array(test_labels)

Build the base model.

In [7]:
base_model= ResNet50(input_shape=(512,768,3), weights='imagenet', include_top=False)
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

for layer in base_model.layers:
    layer.trainable = False

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 16, 24, 2048)      23587712  
                                                                 
 flatten (Flatten)           (None, 786432)            0         
                                                                 
 batch_normalization (BatchN  (None, 786432)           3145728   
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               201326848 
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 256)               0

Configure callbacks.

In [8]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Seminar2/resnet50/',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

es = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=6
)

Train the model.

In [ ]:
model.fit(
    x=X_train,
    y=y_train,
    batch_size=4,
    steps_per_epoch=50,
    epochs=30,
    validation_data=(X_valid, y_valid),
    validation_steps=25,
    callbacks=[es, checkpoint_callback],
    verbose=1
)

Epoch 1/30
50/50 [==============================] - ETA: 0s - loss: -0.2451 - accuracy: 0.4000
Epoch 1: val_accuracy improved from -inf to 0.46000, saving model to /content/drive/MyDrive/Seminar2/resnet50/


In [ ]:
from sklearn import metrics

predictions = model.predict(X_test, verbose=True)
predictions = np.where(predictions > .5, 1, 0)
print(metrics.classification_report(y_test, predictions))